# Contents
### [Generate.py](#Generate.py)
### [stateless.py](#stateless.py)
### [path.py](#path.py)
### [models.py](#models.py)

## Introduction
This notebook should explain the majority of what the different folders and python files in this directory do. The data and initial implementation were built off of arcpy/arcgis so some of these components still exist.

'default.gdb' is a database holding the original 30m resolution heightmap and derived slope and aspect maps for the full area, about 60km x 60km. I believe 'info' is a folder used by arcmap, however none of the python files interact with it directly.

For a similar reason, the arcmap profile "generation.mxd" still exists, allowing you to view the three initial maps if needed.

Some error detection has been added for the most likely errors with files missing etc. which should help identify the cause of some issues.


### Generate.py <a name="Generate.py"></a>

Generate.py is the only python file still using arcpy, however it should only be needed if the area being surveyed changes. Although it has methods with fairly self-descriptive names to generate the necessary data, its main purpose is being able to generate numpy representations of maps for faster use by other programs. 

Assuming the height,slope and aspect maps have been placed in default.gdb with the names "srtm30_dem_utm45n_crop", "Slope_srtm" and "Aspect_srtm" respectively (these are hardcoded in but should be easy to change in the Setup method), this is done as follows:

In [ ]:
import Generate
Generate.storeRasters()

## stateless.py <a name="stateless.py"></a>

This creates the arrays needed to calculate the radar response for a single point. It assumes the info.txt and maps.npz files mentioned previously exist in the same directory.

In [ ]:
import stateless
stateless.Setup() # loads data from info.txt and maps.npz
x,y = 469900.0, 3095000.0
stateless.generateMaps(x,y,"demoPoint")

These are the only methods that should be called from outside the class. The full signature of generateMaps:

generateMaps(x, y, path, above_ground=100.0, isOffset = True, antennaDir = None)

'path' is the directory to save the data in. 'above_ground' specifies the elevation of the radar and 'isOffset' indicates that this is relative to the ground. 'antennaDir' can take an angle to also generate the angle from the surface to the direction the antenna was facing for each point of the surface.

## path.py <a name="path.py"></a>

This will call stateless.py for each point specified by a data file containing the path of the radar. Note, the directory name passed will have 1 subdirectory created per point, no other files or directories should be created within this directory else other methods will attempt to read non-data files as if they were data.

'loadFromFile(filename, outName=None)' takes a custom file format specified in the python file.

'loadGpx(filename, outName=None)' uses data specified in the gpx xml format. If not specified, this will use the name specified in xml as the output folder name and the filename if this also does not exist.

In [ ]:
import path
path.loadGpx("demoPath")

## models.py <a name="models.py"></a>

This generates a plot of the response for either a single point or a series of points. At time of writing, the best model to use is unknown so there are various options for plotting with different models of the response from each surface.

In [ ]:
import models
models.models = [models.lambertian, models.raySpecular, lambda x : models.Minnaert(x,2)]
models.titles = ["diffuse", "specular", "Minnaert (k=2)"]
models.compare("demoPath") # plots a radargram for each model specified in models.models on a single figure

models.wiggle("demoPoint")
models.manyWiggle("demoSparsePath")

The optional arguments allow the wave being modelled, the surface interactions, or how slices in radargrams are aligned to all be altered. All methods currently apply a bandpass filter but this does not have any options to adjust the ranges filtered without setting them in the code.

In [ ]:
compare(name,adjusted=False,wave=GaussianDot())

wiggle(name, intensityModel=raySpecular, wave=GaussianDot())

manyWiggle(name, adjusted=False, intensityModel=raySpecular, wave=GaussianDot())

'name' is the directory data is stored in. 

'adjusted' indicates that points with equal elevation should be aligned i.e. For a radar path of changing elevation over a level surface, the radargram would still be a flat line.

'intensityModel' is the class within models.py to use to model the response from each surface.

'wave' is the waveform to convolve over the responses to produce a radargram or wiggle plot.